<h1>Preprocessing of the Aconity data</h1>
<p>First, the aconity data needs to be processed so that it is easily ingestible by the machine learning model.</p>
<p>This involves</p>
<lo>
<li>Reading and parsing the Aconity data files</li>
<li>Calibrating the data to real units</li>
<li>Masking and labelling the data</li>
<li>Recontructing a time column from the indeces</li>
<li>Saving the processed data for later use in the experiment</li>
</lo>

In [1]:
from MTPy.meltpool_tomography import MeltpoolTomography
from dask.distributed import Client, LocalCluster
import pickle
from tqdm.auto import tqdm

In [2]:
# Prepare a dask cluster and client
cluster = LocalCluster(n_workers=12, threads_per_worker=1)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:34089/status,
Dashboard: http://127.0.0.1:34089/status,Workers: 12
Total threads: 12,Total memory: 15.27 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43953,Workers: 12
Dashboard: http://127.0.0.1:34089/status,Total threads: 12
Started: Just now,Total memory: 15.27 GiB
Comm: tcp://127.0.0.1:42697,Total threads: 1
Dashboard: http://127.0.0.1:39477/status,Memory: 1.27 GiB
Nanny: tcp://127.0.0.1:46605,


In [3]:
# Create an MTPy object with the client
mpt = MeltpoolTomography(client=client)

In [ ]:
# Read the aconity data
mpt.read_layers("data/pyrometry_data")

In [ ]:
# Examine the raw data
raw_plot = mpt.scatter2d()
raw_plot

In [2]:
# Import the pre-prepared masks
with open("data/masks2.pkl", "rb") as f:
    masks = pickle.load(f)

In [ ]:
# Apply the masks
mpt.rotate_xy(-45.0)
mpt.mask_xyrectangles(masks)
mpt.rotate_xy(45.0)

In [ ]:
# Compare the raw and masked data
mpt.scatter2d() + raw_plot

In [9]:
# Add a time column to the data
delta_time = 0.0001  # 10kHz sampling rate
mpt.data["time"] = mpt.data.index * delta_time

In [ ]:
# Save data for use in the experiment
mpt.save("checkpoint.mtp")

In [7]:
mpt.data

,x,y,z,t,sample,time
npartitions=46,,,,,,
,float64,float64,float64,float64,int64,float64
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [8]:
samples = mpt.data["sample"].unique().compute()

for future in tqdm(
    (
        mpt.data.loc[mpt.data["sample"] == sample_num]
        .drop("sample", axis=1)
        .astype("float32")
        .repartition(
            partition_size="10MB"
        )  # Break up into small chunks ingestible by an ML model
        .to_parquet(
            f"sample_X/{sample_num}",
            compression="lz4",
            write_metadata_file=True,
            compute=False,
        )
        for sample_num in samples
    ),
    total=len(samples),
):
    future.compute()

  0%|          | 0/81 [00:00<?, ?it/s]

Task exception was never retrieved
future: <Task finished name='Task-1318357' coro=<Client._gather.<locals>.wait() done, defined at /home/cianh/Programming/Git_Projects/Aconity_ML_Test/.venv/lib/python3.11/site-packages/distributed/client.py:2212> exception=AllExit()>
Traceback (most recent call last):
  File "/home/cianh/Programming/Git_Projects/Aconity_ML_Test/.venv/lib/python3.11/site-packages/distributed/client.py", line 2221, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-1318370' coro=<Client._gather.<locals>.wait() done, defined at /home/cianh/Programming/Git_Projects/Aconity_ML_Test/.venv/lib/python3.11/site-packages/distributed/client.py:2212> exception=AllExit()>
Traceback (most recent call last):
  File "/home/cianh/Programming/Git_Projects/Aconity_ML_Test/.venv/lib/python3.11/site-packages/distributed/client.py", line 2221, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was n

KeyboardInterrupt: 